# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    print('limit:',limit)
    while n <= limit:
        yield n ** 0.5
        if (n==13):
            print('13th number yielded',sqrt_value)
        n += 1

# Example usage:
limit = 1300
generator = square_root_generator(limit)
sum = 0

for sqrt_value in generator:
    sum += sqrt_value
    print(sqrt_value)    
    
print('Sum:',sum)


limit: 1300
1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989
13th number yielded 3.605551275463989
3.7416573867739413
3.872983346207417
4.0
4.123105625617661
4.242640687119285
4.358898943540674
4.47213595499958
4.58257569495584
4.69041575982343
4.795831523312719
4.898979485566356
5.0
5.0990195135927845
5.196152422706632
5.291502622129181
5.385164807134504
5.477225575051661
5.5677643628300215
5.656854249492381
5.744562646538029
5.830951894845301
5.916079783099616
6.0
6.082762530298219
6.164414002968976
6.244997998398398
6.324555320336759
6.4031242374328485
6.48074069840786
6.557438524302
6.6332495807108
6.708203932499369
6.782329983125268
6.855654600401044
6.928203230275509
7.0
7.0710678118654755
7.14142842854285
7.211102550927978
7.280109889280518
7.3484692283495345
7.416198487095663
7.483314773547883
7.54983443527075
7.615773105863909
7.6811

# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [2]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [3]:
import dlt

# define the connection to load to.
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='wk1')


# load 
info = generators_pipeline.run(people_1(),
										table_name="lk_people",
										write_disposition="replace")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

# load 
info = generators_pipeline.run(people_2(),
										table_name="lk_people",
										write_disposition="append")


Pipeline dlt_ipykernel_launcher load step completed in 0.32 seconds
1 load package(s) were loaded to destination duckdb and into dataset wk1
The duckdb destination used duckdb:///C:\Git\courses\data-engineering-zoomcamp\cohorts\2024\workshops\dlt_resources\dlt_ipykernel_launcher.duckdb location to store data
Load package 1708259251.9464118 is LOADED and contains no failed jobs


In [4]:
# show outcome

import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n http_download table below:")

people = conn.sql("SELECT * FROM lk_people").df()
display(people)

print("\n\n\n stream_download table below:")

sum_ages = conn.sql("SELECT sum(age) FROM lk_people ").df()
display(sum_ages)


# As you can see, the same data was loaded in both cases.

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ lk_people           │
│ lk_people_with_pk   │
│ lk_people_with_pk   │
│ t1                  │
└─────────────────────┘




 http_download table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708259251.9464118,q/FJ8gnhrIm/Ig,None
1,2,Person_2,27,City_A,1708259251.9464118,2VjJCQME41oZXw,None
2,3,Person_3,28,City_A,1708259251.9464118,c6/knGfQSq+Y0g,None
3,4,Person_4,29,City_A,1708259251.9464118,JAL0Uy4zTnmXGw,None
4,5,Person_5,30,City_A,1708259251.9464118,2GOSBp0bnf/ZCQ,None
5,3,Person_3,33,City_B,1708259252.717781,cNcSc3ygYsuDDQ,Job_3
6,4,Person_4,34,City_B,1708259252.717781,9md4LqXfOa1F0w,Job_4
7,5,Person_5,35,City_B,1708259252.717781,B4vF0GBZl5a0PA,Job_5
8,6,Person_6,36,City_B,1708259252.717781,ooYuuqdtgYQDkQ,Job_6
9,7,Person_7,37,City_B,1708259252.717781,KCML+JpI2bHNDA,Job_7





 stream_download table below:


,sum(age)
0,353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [5]:
import dlt

# define the connection to load to.
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='wk1')


# load 
info = generators_pipeline.run(people_1(),
										table_name="lk_people_with_pk",
                                        primary_key="id",
										write_disposition="replace")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

# load 
info = generators_pipeline.run(people_2(),
										table_name="lk_people_with_pk",
										write_disposition="merge")

Pipeline dlt_ipykernel_launcher load step completed in 0.21 seconds
1 load package(s) were loaded to destination duckdb and into dataset wk1
The duckdb destination used duckdb:///C:\Git\courses\data-engineering-zoomcamp\cohorts\2024\workshops\dlt_resources\dlt_ipykernel_launcher.duckdb location to store data
Load package 1708259253.5839422 is LOADED and contains no failed jobs


In [6]:
print("\n\n\n http_download table below:")

people = conn.sql("SELECT * FROM lk_people_with_pk").df()
display(people)

sum_ages = conn.sql("SELECT sum(age) FROM lk_people_with_pk").df()
display(sum_ages)





 http_download table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708259253.5839422,KRJdSdscH+6fzA,None
1,2,Person_2,27,City_A,1708259253.5839422,U4SD6cKjvhDqCg,None
2,3,Person_3,33,City_B,1708259254.2427714,y9pBBBAqcGxM+g,Job_3
3,6,Person_6,36,City_B,1708259254.2427714,8FTfwv1G06Qw6w,Job_6
4,5,Person_5,35,City_B,1708259254.2427714,Tfc8rSMJ0PoGxA,Job_5
5,8,Person_8,38,City_B,1708259254.2427714,O0izanh0iCnQew,Job_8
6,4,Person_4,34,City_B,1708259254.2427714,lgk4vFoNweb57g,Job_4
7,7,Person_7,37,City_B,1708259254.2427714,DRCb2POWdsC/Vw,Job_7


,sum(age)
0,266.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX